# Ames Housing Prices - Step 5: Model Deployment
Now that we have trained and selected our optimal model, its time to deploy it.  This notebook demonstrates how to user our Experiment and Pipelines from the previous steps to easly deploy our model as a Cortex Action. 

In [1]:
# Basic setup
%run config.ipynb

Cortex Python SDK v5.4a5


In [4]:
# Connect to Cortex 5 and create a Builder instance
cortex = Cortex.client()
builder = cortex.builder()

### Load the Experiement
Let's load our experiment from the previous step and find the model we want to deploy.

In [5]:
exp = cortex.experiment('kaggle/ames-housing-regression')
exp

---
The model created in the last run looks to be the best, let's deploy it

In [6]:
run = exp.get_run('sb04akg')
model = run.get_artifact('model')
model

LassoCV(alphas=[1, 0.1, 0.001, 0.0005], copy_X=True, cv=None, eps=0.001,
    fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

### Model deployment - Step 1: Configure Data Pipeline for Inputs
Our model was trained with data that has had cleaning and feature engineering steps applied to it.  Since we want our users to send us the actual raw data, we need to deploy our pipeline to transform the input data into the form we expect.  This requires applying some of the same steps from before, but also requires us to remember some of the data created during model training such as the median values of certain columns and the final list of _dummy_ categorical columns created during feature engineering.  Luckily, our pipelines have a memory in the form of _context_ that we can reference here to achieve this.

In [8]:
train_ds = cortex.dataset('kaggle/ames-housing-train')

# Model our feature pipeline after the 'clean' pipeline
x_pipe = builder.pipeline('x_pipe')
x_pipe.from_pipeline(train_ds.pipeline('clean'))

In [9]:
# Same idea from our training prep, however we need to use the median values we computed before which we stored in our pipeline context
def fill_median_cols_ctx(pipeline, df):
    fill_median_cols = ['GarageArea','TotalBsmtSF', 'MasVnrArea', 'BsmtFinSF1', 'LotFrontage', 'BsmtUnfSF', 'GarageYrBlt']
    [df[j].fillna(pipeline.get_context('{}_median'.format(j)), inplace=True) for j in fill_median_cols]
                  
# The dummy column conversion we did during training needs to be applied here.  Afterwards there will be missing columns because 
# our input instance will only contain at most one value per category.  We need to fill in the other expected columns.  We stored
# the expected set of columns in our pipeline so we can easily do this now.
def fix_columns(pipeline, df):
    all_cols = pipeline.get_context('columns')
    missing_cols = set(all_cols) - set(df.columns)
    for c in missing_cols:
        df[c] = 0
    
    # make sure we have all the columns we need
    assert(set(all_cols) - set(df.columns) == set())
    
    return df[all_cols]

In [10]:
# The feature engineering pipeline contains the complete list of dummy columns in addition to some steps we need
engineer_pipe = train_ds.pipeline('engineer')
x_pipe.set_context('columns', engineer_pipe.get_context('columns'))

# Reuse steps from our clean, features, and engineer pipelines
fill_zero_cols = x_pipe.get_step('fill_zero_cols')
fill_na_none = x_pipe.get_step('fill_na_none')
get_dummies = engineer_pipe.get_step('get_dummies')

# Build our final input pipeline
x_pipe.reset()
x_pipe.add_step(fill_zero_cols)
x_pipe.add_step(fill_median_cols_ctx)
x_pipe.add_step(fill_na_none)
x_pipe.add_step(get_dummies)
x_pipe.add_step(fix_columns)

### Model deployment - Step 2: Configure Data Pipeline for Output
If you remember, we scaled our target variable using the numpy _log1p_ function.  We need to inverse this using the _exp_ function so our predicted value is correct.

In [12]:
y_pipe = builder.pipeline('y_pipe')

In [14]:
def rescale_target(pipeline, df):
    df['SalePrice'] = np.exp(df['SalePrice'])

In [15]:
y_pipe.add_step(rescale_target)

### Model deployment - Step 3: Build and Deploy Cortex Action
Now that we have our input and output pipelines, we can use the Cortex Builder to package and deploy our model in one step.

In [21]:
builder.action('mattsanchez/ames-housing-predict')\
       .from_model(model, x_pipeline=x_pipe, y_pipeline=y_pipe, target='SalePrice')\
       .image_prefix('registry.cortex-dev.insights.ai:5000')\
       .build()

Building Cortex Action (function): mattsanchez/ames-housing-predict
Building Docker image mattsanchez/ames-housing-predict:4qada7w...
Step 1/11 : FROM continuumio/miniconda3:4.5.4
Step 2/11 : WORKDIR /function
Step 3/11 : RUN apt-get update && apt-get install -y linux-headers-amd64 build-essential
Step 4/11 : RUN conda config --add channels conda-forge
Step 5/11 : COPY conda_requirements.txt .
Step 6/11 : RUN conda install --yes --file conda_requirements.txt
Step 7/11 : RUN pip install "dill==0.2.8.2" "fdk==0.0.31" "cortex-client==5.4a5"
Running setup.py bdist_wheel for dill: started
Running setup.py bdist_wheel for dill: finished with status 'done'
Stored in directory: /root/.cache/pip/wheels/e2/5d/17/f87cb7751896ac629b435a8696f83ee75b11029f5d6f6bda72
Running setup.py bdist_wheel for cortex-client: started
Running setup.py bdist_wheel for cortex-client: finished with status 'done'
Stored in directory: /root/.cache/pip/wheels/0b/df/b4/109ce0f29485629876789698082abe7340e968ecd6bf7471c6


Name,Version,Kind,Image,Deployment Status,mattsanchez/ames-housing-predict,6,python:3,registry.cortex-dev.insights.ai:5000/mattsanchez/ames-housing-predict:4qada7w,COMPLETED


In [33]:
action = cortex.action('mattsanchez/ames-housing-predict')
action

Name,Version,Kind,Image,Deployment Status,mattsanchez/ames-housing-predict,6,python:3,registry.cortex-dev.insights.ai:5000/mattsanchez/ames-housing-predict:4qada7w,COMPLETED


---
Unit test for the Action.  Make sure our action is ready for use.

In [43]:
%%time

params = {
    "columns": ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition'],
    "values": [[20,"RH",80.0,11622,"Pave",None,"Reg","Lvl","AllPub","Inside","Gtl","NAmes","Feedr","Norm","1Fam","1Story",5,6,1961,1961,"Gable","CompShg","VinylSd","VinylSd","None",0.0,"TA","TA","CBlock","TA","TA","No","Rec",468.0,"LwQ",144.0,270.0,882.0,"GasA","TA","Y","SBrkr",896,0,0,896,0.0,0.0,1,0,2,1,"TA",5,"Typ",0,None,"Attchd",1961.0,"Unf",1.0,730.0,"TA","TA","Y",140,0,0,0,120,0,None,"MnPrv",None,0,6,2010,"WD","Normal"]]
}

result = action.invoke(message=Message.with_payload(params))
print(result.payload)
print()

{'columns': ['SalePrice'], 'values': [120665.4689426633]}

CPU times: user 17.6 ms, sys: 1.94 ms, total: 19.6 ms
Wall time: 559 ms


## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  We start by creating a Schema that defines our input for Ames Housing price prediction.  The schema will be built automatically using the parameters we already defined in our training dataset.

In [38]:
x_schema = builder.schema('kaggle/ames-housing-instance').title('Ames Housing Test Instance').from_parameters(train_ds.parameters[1:][:-1]).build()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Ames Housing Price Prediction" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.

In [39]:
b = builder.skill('kaggle/ames-housing-price-predict').title('Ames Housing Price Prediction').description('Predicts the price of a houses in Ames, Iowa.')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action that we built previously to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [41]:
b = b.input('ames-house').title('Ames House').use_schema(x_schema.name).all_routing(action, 'price-prediction').build()

In the previous step, we referenced an Output called **price-prediction**.  We can create that Output here using the Output sub-builder.

In [42]:
b = b.output('price-prediction').title('Price Prediction').parameter(name='SalePrice', type='number', format='double').build()

We can preview the CAMEL document our builder will create to make sure everything looks correct.

In [43]:
b.to_camel()

{'camel': '1.0.0',
 'name': 'kaggle/ames-housing-price-predict',
 'title': 'Ames Housing Price Prediction',
 'inputs': [{'name': 'ames-house',
   'title': 'Ames House',
   'parameters': {'$ref': 'kaggle/ames-housing-instance'},
   'routing': {'all': {'action': 'kaggle/ames-housing-predict',
     'output': 'price-prediction'}}}],
 'outputs': [{'name': 'price-prediction',
   'title': 'Price Prediction',
   'parameters': [{'name': 'SalePrice',
     'type': 'number',
     'required': True,
     'format': 'double'}]}],
 'description': 'Predicts the price of a houses in Ames, Iowa.'}

---
### Build and Publish the Skill to the Marketplace
This will build the Skill and publish it to my private marketplace.  It will then be available for use in the Agent Builder.

In [45]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

Ames Housing Price Prediction (kaggle/ames-housing-price-predict) v1
